# Imports

In [1]:
import math
import pandas as pd
import pennylane as qml
import time

from keras.datasets import mnist
from matplotlib import pyplot as plt
from pennylane import numpy as np
from pennylane.templates import AmplitudeEmbedding, AngleEmbedding
from pennylane.templates.subroutines import ArbitraryUnitary
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Model Params

In [2]:
np.random.seed(131)
initial_params = np.random.random([19])

INITIALIZATION_METHOD = 'Amplitude'
BATCH_SIZE = 20
EPOCHS = 400

STEP_SIZE = 0.01
BETA_1 = 0.9
BETA_2 = 0.99
EPSILON = 0.00000001

TRAINING_SIZE = 0.78
VALIDATION_SIZE = 0.07
TEST_SIZE = 1-TRAINING_SIZE-VALIDATION_SIZE

initial_time = time.time()

# Import dataset

In [3]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()
examples = np.append(train_X, test_X, axis=0)
examples = examples.reshape(70000, 28*28)
classes = np.append(train_y, test_y)

In [4]:
x = []
y = []
for (example, label) in zip(examples, classes):
    if label in [0, 2, 4, 6, 8]:
        x.append(example)
        y.append(-1)
    else:
        x.append(example)
        y.append(1)

In [5]:
x = np.array(x)
y = np.array(y)

# Normalize pixels values
x = x / 255

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=TEST_SIZE, shuffle=True)

In [6]:
validation_indexes = np.random.random_integers(len(X_train), size=(math.floor(len(X_train)*VALIDATION_SIZE),))
X_validation = [X_train[n-1] for n in validation_indexes]
y_validation = [y_train[n-1] for n in validation_indexes]

In [7]:
pca = PCA(n_components=8)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_validation = pca.transform(X_validation)
X_test = pca.transform(X_test)

preprocessing_time = time.time()

# Circuit creation

In [8]:
device = qml.device("default.qubit", wires=3)

In [9]:
def unitary(params, wire1, wire2):
    # qml.RZ(0, wires=wire1)
    qml.RY(params[0], wires=wire1)
    # qml.RZ(0, wires=wire1)
    # qml.RZ(0, wires=wire2)
    qml.RY(params[1], wires=wire2)
    # qml.RZ(0, wires=wire2)
    qml.CNOT(wires=[wire2, wire1])
    # qml.RZ(0, wires=wire1)
    qml.RY(params[2], wires=wire2)
    qml.CNOT(wires=[wire1, wire2])
    qml.RY(params[3], wires=wire2)
    qml.CNOT(wires=[wire2, wire1])
    # qml.RZ(0, wires=wire1)
    qml.RY(params[4], wires=wire1)
    # qml.RZ(0, wires=wire1)
    # qml.RZ(0, wires=wire2)
    qml.RY(params[5], wires=wire2)
    # qml.RZ(0, wires=wire2)

In [10]:
@qml.qnode(device)
def circuit(features, params):
    # Load state
    if INITIALIZATION_METHOD == 'Amplitude':
        AmplitudeEmbedding(features=features, wires=range(3), normalize=True, pad_with=0.)
    else:
        AngleEmbedding(features=features, wires=range(3), rotation='Y')

    # First layer
    unitary(params[:6], 1, 2)

    # Second layer
    unitary(params[6:12], 0, 1)

    # Third layer
    unitary(params[12:18], 1, 2)

    # Fourth layer
    qml.RY(params[18], wires=2)

    # Measurement
    return qml.expval(qml.PauliZ(2))

## Circuit example

In [11]:
features = X_train[0]
print(f"Inital parameters: {initial_params}\n")
print(f"Example features: {features}\n")
print(f"Expectation value: {circuit(features, initial_params)}\n")
print(circuit.draw())

Inital parameters: [0.65015361 0.94810917 0.38802889 0.64129616 0.69051205 0.12660931
 0.23946678 0.25415707 0.42644165 0.83900255 0.74503365 0.38067928
 0.26169292 0.05333379 0.43689638 0.20897912 0.59441102 0.09890353
 0.22409353]

Example features: [-2.1413895  -1.91460434  2.14827728  0.87541673  0.96048781  2.90317527
 -2.82610656  0.26743871]

Expectation value: 0.35090291716142186

 0: ──╭QubitStateVector(M0)──RY(0.239)───────────────────────────────────────────────────────────╭X─────────────╭C─────────────╭X──RY(0.745)──────────────────────────────────────────────────────────────────────┤     
 1: ──├QubitStateVector(M0)──RY(0.65)───╭X─────────────╭C─────────────╭X──RY(0.691)──RY(0.254)───╰C──RY(0.426)──╰X──RY(0.839)──╰C──RY(0.381)──RY(0.262)──╭X─────────────╭C─────────────╭X──RY(0.594)──────────────┤     
 2: ──╰QubitStateVector(M0)──RY(0.948)──╰C──RY(0.388)──╰X──RY(0.641)──╰C──RY(0.127)──RY(0.0533)──────────────────────────────────────────────────────────╰C──RY(0.437)──╰X──RY

# Accuracy test definition

In [12]:
def measure_accuracy(x, y, circuit_params):
    class_errors = 0

    for example, example_class in zip(x, y):
        predicted_value = circuit(example, circuit_params)

        if (example_class > 0 and predicted_value <= 0) or (example_class <= 0 and predicted_value > 0):
            class_errors += 1

    return 1 - (class_errors/len(y))

# Training

In [13]:
params = initial_params
opt = qml.AdamOptimizer(stepsize=STEP_SIZE, beta1=BETA_1, beta2=BETA_2, eps=EPSILON)
test_accuracies = []
best_validation_accuracy = 0.0
best_params = []

for i in range(len(X_train)):
    features = X_train[i]
    expected_value = y_train[i]

    def cost(circuit_params):
        value = circuit(features, circuit_params)
        return ((expected_value - value) ** 2)/len(X_train)

    params = opt.step(cost, params)

    if i % BATCH_SIZE == 0:
        print(f"epoch {i//BATCH_SIZE}")

    if i % (10*BATCH_SIZE) == 0:
        current_accuracy = measure_accuracy(X_validation, y_validation, params)
        test_accuracies.append(current_accuracy)
        print(f"accuracy: {current_accuracy}")

        if current_accuracy > best_validation_accuracy:
            print("best accuracy so far!")
            best_validation_accuracy = current_accuracy
            best_params = params

    if len(test_accuracies) == 30:
        print(f"test_accuracies: {test_accuracies}")

        if np.allclose(best_validation_accuracy, test_accuracies[0]):
            params = best_params
            break

        del test_accuracies[0]

epoch 0
accuracy: 0.4633853541416567
best accuracy so far!
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
accuracy: 0.5356542617046819
best accuracy so far!
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
accuracy: 0.5863145258103242
best accuracy so far!
epoch 21
epoch 22
epoch 23
epoch 24
epoch 25
epoch 26
epoch 27
epoch 28
epoch 29
epoch 30
accuracy: 0.6110444177671068
best accuracy so far!
epoch 31
epoch 32
epoch 33
epoch 34
epoch 35
epoch 36
epoch 37
epoch 38
epoch 39
epoch 40
accuracy: 0.646578631452581
best accuracy so far!
epoch 41
epoch 42
epoch 43
epoch 44
epoch 45
epoch 46
epoch 47
epoch 48
epoch 49
epoch 50
accuracy: 0.6645858343337334
best accuracy so far!
epoch 51
epoch 52
epoch 53
epoch 54
epoch 55
epoch 56
epoch 57
epoch 58
epoch 59
epoch 60
accuracy: 0.680672268907563
best accuracy so far!
epoch 61
epoch 62
epoch 63
epoch 64
epoch 65
epoch 66
epoch 67
epoch 68
epoch 69
epoch 70
accuracy: 0.6564

epoch 341
epoch 342
epoch 343
epoch 344
epoch 345
epoch 346
epoch 347
epoch 348
epoch 349
epoch 350
accuracy: 0.7056422569027612
test_accuracies: [0.680672268907563, 0.6564225690276111, 0.6725090036014405, 0.6761104441776711, 0.6849939975990396, 0.6653061224489796, 0.6689075630252102, 0.6895558223289315, 0.6991596638655462, 0.7001200480192077, 0.6830732292917168, 0.7008403361344537, 0.6957983193277311, 0.7015606242496999, 0.702280912364946, 0.694357743097239, 0.6931572629051621, 0.692436974789916, 0.7097238895558223, 0.6960384153661465, 0.6936374549819928, 0.7188475390156062, 0.7066026410564226, 0.7027611044417768, 0.7042016806722688, 0.7042016806722688, 0.6941176470588235, 0.6948379351740697, 0.6866746698679471, 0.7056422569027612]
epoch 351
epoch 352
epoch 353
epoch 354
epoch 355
epoch 356
epoch 357
epoch 358
epoch 359
epoch 360
accuracy: 0.7114045618247299
test_accuracies: [0.6564225690276111, 0.6725090036014405, 0.6761104441776711, 0.6849939975990396, 0.6653061224489796, 0.66890756

epoch 451
epoch 452
epoch 453
epoch 454
epoch 455
epoch 456
epoch 457
epoch 458
epoch 459
epoch 460
accuracy: 0.6996398559423769
test_accuracies: [0.7008403361344537, 0.6957983193277311, 0.7015606242496999, 0.702280912364946, 0.694357743097239, 0.6931572629051621, 0.692436974789916, 0.7097238895558223, 0.6960384153661465, 0.6936374549819928, 0.7188475390156062, 0.7066026410564226, 0.7027611044417768, 0.7042016806722688, 0.7042016806722688, 0.6941176470588235, 0.6948379351740697, 0.6866746698679471, 0.7056422569027612, 0.7114045618247299, 0.7210084033613445, 0.7092436974789915, 0.6979591836734693, 0.7133253301320528, 0.7198079231692678, 0.7118847539015607, 0.7198079231692678, 0.7121248499399759, 0.7099639855942377, 0.6996398559423769]
epoch 461
epoch 462
epoch 463
epoch 464
epoch 465
epoch 466
epoch 467
epoch 468
epoch 469
epoch 470
accuracy: 0.6785114045618248
test_accuracies: [0.6957983193277311, 0.7015606242496999, 0.702280912364946, 0.694357743097239, 0.6931572629051621, 0.692436974

epoch 561
epoch 562
epoch 563
epoch 564
epoch 565
epoch 566
epoch 567
epoch 568
epoch 569
epoch 570
accuracy: 0.7116446578631452
test_accuracies: [0.7066026410564226, 0.7027611044417768, 0.7042016806722688, 0.7042016806722688, 0.6941176470588235, 0.6948379351740697, 0.6866746698679471, 0.7056422569027612, 0.7114045618247299, 0.7210084033613445, 0.7092436974789915, 0.6979591836734693, 0.7133253301320528, 0.7198079231692678, 0.7118847539015607, 0.7198079231692678, 0.7121248499399759, 0.7099639855942377, 0.6996398559423769, 0.6785114045618248, 0.709483793517407, 0.7217286914765906, 0.7210084033613445, 0.726530612244898, 0.7207683073229292, 0.7044417767106843, 0.7171668667466986, 0.7106842737094838, 0.7135654261704683, 0.7116446578631452]
epoch 571
epoch 572
epoch 573
epoch 574
epoch 575
epoch 576
epoch 577
epoch 578
epoch 579
epoch 580
accuracy: 0.6804321728691476
test_accuracies: [0.7027611044417768, 0.7042016806722688, 0.7042016806722688, 0.6941176470588235, 0.6948379351740697, 0.686674

epoch 672
epoch 673
epoch 674
epoch 675
epoch 676
epoch 677
epoch 678
epoch 679
epoch 680
accuracy: 0.7063625450180072
test_accuracies: [0.6979591836734693, 0.7133253301320528, 0.7198079231692678, 0.7118847539015607, 0.7198079231692678, 0.7121248499399759, 0.7099639855942377, 0.6996398559423769, 0.6785114045618248, 0.709483793517407, 0.7217286914765906, 0.7210084033613445, 0.726530612244898, 0.7207683073229292, 0.7044417767106843, 0.7171668667466986, 0.7106842737094838, 0.7135654261704683, 0.7116446578631452, 0.6804321728691476, 0.7234093637454981, 0.7142857142857143, 0.7188475390156062, 0.720048019207683, 0.7253301320528212, 0.7222088835534214, 0.715486194477791, 0.7058823529411764, 0.6986794717887155, 0.7063625450180072]
epoch 681
epoch 682
epoch 683
epoch 684
epoch 685
epoch 686
epoch 687
epoch 688
epoch 689
epoch 690
accuracy: 0.7145258103241297
test_accuracies: [0.7133253301320528, 0.7198079231692678, 0.7118847539015607, 0.7198079231692678, 0.7121248499399759, 0.7099639855942377, 

epoch 782
epoch 783
epoch 784
epoch 785
epoch 786
epoch 787
epoch 788
epoch 789
epoch 790
accuracy: 0.7078031212484994
test_accuracies: [0.7210084033613445, 0.726530612244898, 0.7207683073229292, 0.7044417767106843, 0.7171668667466986, 0.7106842737094838, 0.7135654261704683, 0.7116446578631452, 0.6804321728691476, 0.7234093637454981, 0.7142857142857143, 0.7188475390156062, 0.720048019207683, 0.7253301320528212, 0.7222088835534214, 0.715486194477791, 0.7058823529411764, 0.6986794717887155, 0.7063625450180072, 0.7145258103241297, 0.7190876350540216, 0.7253301320528212, 0.7116446578631452, 0.7111644657863145, 0.7164465786314526, 0.7272509003601441, 0.7130852340936374, 0.7130852340936374, 0.7020408163265306, 0.7078031212484994]
epoch 791
epoch 792
epoch 793
epoch 794
epoch 795
epoch 796
epoch 797
epoch 798
epoch 799
epoch 800
accuracy: 0.7135654261704683
test_accuracies: [0.726530612244898, 0.7207683073229292, 0.7044417767106843, 0.7171668667466986, 0.7106842737094838, 0.7135654261704683, 

epoch 891
epoch 892
epoch 893
epoch 894
epoch 895
epoch 896
epoch 897
epoch 898
epoch 899
epoch 900
accuracy: 0.7126050420168067
test_accuracies: [0.7188475390156062, 0.720048019207683, 0.7253301320528212, 0.7222088835534214, 0.715486194477791, 0.7058823529411764, 0.6986794717887155, 0.7063625450180072, 0.7145258103241297, 0.7190876350540216, 0.7253301320528212, 0.7116446578631452, 0.7111644657863145, 0.7164465786314526, 0.7272509003601441, 0.7130852340936374, 0.7130852340936374, 0.7020408163265306, 0.7078031212484994, 0.7135654261704683, 0.7054021608643457, 0.7183673469387755, 0.7162064825930372, 0.7162064825930372, 0.6993997599039616, 0.709483793517407, 0.7104441776710684, 0.7330132052821128, 0.7253301320528212, 0.7126050420168067]
epoch 901
epoch 902
epoch 903
epoch 904
epoch 905
epoch 906
epoch 907
epoch 908
epoch 909
epoch 910
accuracy: 0.7190876350540216
test_accuracies: [0.720048019207683, 0.7253301320528212, 0.7222088835534214, 0.715486194477791, 0.7058823529411764, 0.698679471

epoch 1002
epoch 1003
epoch 1004
epoch 1005
epoch 1006
epoch 1007
epoch 1008
epoch 1009
epoch 1010
accuracy: 0.7128451380552221
test_accuracies: [0.7116446578631452, 0.7111644657863145, 0.7164465786314526, 0.7272509003601441, 0.7130852340936374, 0.7130852340936374, 0.7020408163265306, 0.7078031212484994, 0.7135654261704683, 0.7054021608643457, 0.7183673469387755, 0.7162064825930372, 0.7162064825930372, 0.6993997599039616, 0.709483793517407, 0.7104441776710684, 0.7330132052821128, 0.7253301320528212, 0.7126050420168067, 0.7190876350540216, 0.7015606242496999, 0.7123649459783914, 0.719327731092437, 0.7056422569027612, 0.7212484993997599, 0.7159663865546219, 0.7186074429771909, 0.7176470588235294, 0.704921968787515, 0.7128451380552221]
epoch 1011
epoch 1012
epoch 1013
epoch 1014
epoch 1015
epoch 1016
epoch 1017
epoch 1018
epoch 1019
epoch 1020
accuracy: 0.7166866746698679
test_accuracies: [0.7111644657863145, 0.7164465786314526, 0.7272509003601441, 0.7130852340936374, 0.7130852340936374, 

epoch 1111
epoch 1112
epoch 1113
epoch 1114
epoch 1115
epoch 1116
epoch 1117
epoch 1118
epoch 1119
epoch 1120
accuracy: 0.7135654261704683
test_accuracies: [0.7162064825930372, 0.7162064825930372, 0.6993997599039616, 0.709483793517407, 0.7104441776710684, 0.7330132052821128, 0.7253301320528212, 0.7126050420168067, 0.7190876350540216, 0.7015606242496999, 0.7123649459783914, 0.719327731092437, 0.7056422569027612, 0.7212484993997599, 0.7159663865546219, 0.7186074429771909, 0.7176470588235294, 0.704921968787515, 0.7128451380552221, 0.7166866746698679, 0.7226890756302521, 0.7217286914765906, 0.6684273709483793, 0.7092436974789915, 0.7150060024009603, 0.7056422569027612, 0.7063625450180072, 0.7037214885954381, 0.7133253301320528, 0.7135654261704683]
epoch 1121
epoch 1122
epoch 1123
epoch 1124
epoch 1125
epoch 1126
epoch 1127
epoch 1128
epoch 1129
epoch 1130
accuracy: 0.729171668667467
test_accuracies: [0.7162064825930372, 0.6993997599039616, 0.709483793517407, 0.7104441776710684, 0.733013205

In [14]:
print("Optimized rotation angles: {}".format(params))

training_time = time.time()

Optimized rotation angles: [ 0.38552468  1.95192696  2.19529225  2.61092007  0.66000389  1.24223504
 -1.04831864  0.2236489   0.80316638  1.53137612  0.74503365 -0.0530656
 -0.17205196  1.16895953 -0.71068796 -0.60673554  0.59441102  0.45251568
  0.57770569]


# Testing

In [15]:
accuracy = measure_accuracy(X_test, y_test, params)
print(accuracy)

test_time = time.time()

0.7353333333333334


In [16]:
print(f"pre-processing time: {preprocessing_time-initial_time}")
print(f"training time: {training_time - preprocessing_time}")
print(f"test time: {test_time - training_time}")
print(f"total time: {test_time - initial_time}")

pre-processing time: 14.160719394683838
training time: 4652.954644680023
test time: 78.4348578453064
total time: 4745.550221920013
